<a href="https://colab.research.google.com/github/vkth7/AIFFEL-exploration/blob/main/exp-06/%5Bexp_06%5D_%EC%9E%91%EB%AC%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
import tensorflow as tf
import glob
import os, re
from sklearn.model_selection import train_test_split

In [43]:
txt_file_path = '/content/drive/MyDrive/Colab Notebooks/exp-06/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Well, summer slipped us underneath her tongue', 'Our days and nights are perfumed with obsession', 'Half of my wardrobe is on your bedroom floor']


In [44]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [47]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    # if len(sentence) == 0 | len(sentence) > 15: continue
    if len(sentence) == 0: continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> well , summer slipped us underneath her tongue <end>',
 '<start> our days and nights are perfumed with obsession <end>',
 '<start> half of my wardrobe is on your bedroom floor <end>',
 '<start> use our eyes , throw our hands overboard i am your sweetheart psychopathic crush <end>',
 '<start> drink up your movements , still i can t get enough <end>',
 '<start> i overthink your p punctuation use <end>',
 '<start> not my fault , just a thing that my mind do a rush at the beginning <end>',
 '<start> i get caught up , just for a minute <end>',
 '<start> but lover , you re the one to blame , all that you re doing <end>',
 '<start> can you hear the violence ? <end>']

In [59]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)
print(tensor[:3, :10])

[[   2  136    4 ...    0    0    0]
 [   2  155  377 ...    0    0    0]
 [   2  536   19 ...    0    0    0]
 ...
 [   2 7442    6 ...    0    0    0]
 [   2   67    5 ... 7057    3    0]
 [   2    8    7 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fc10e282c50>
[[   2  136    4  584 3126  131 1299   69  970    3]
 [   2  155  377    8  858   77 9571   31 8552    3]
 [   2  536   19   13 5344   26   18   21 1473  363]]


In [60]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [61]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2  136    4  584 3126  131 1299   69  970    3    0    0    0    0]
[ 136    4  584 3126  131 1299   69  970    3    0    0    0    0    0]


In [62]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,tgt_input,test_size=0.2,random_state=2022 )

In [63]:
len(src_input)

175986

In [64]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
test_dataset = dataset.shuffle(BUFFER_SIZE)
test_dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [109]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size, hidden_size2, hidden_size3):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size2, return_sequences=True)
        self.rnn_3 = tf.keras.layers.LSTM(hidden_size3, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 512
hidden_size2 = 1024
hidden_size3 = 256
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size, hidden_size2, hidden_size3)

In [110]:
optimizer = tf.keras.optimizers.Adam()
#Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)

# model.fit() 함수에 들어가는 다양한 인자를 알고 싶다면 아래의 문서를 참고하세요. 
# https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit
model.fit(enc_train, dec_train, epochs=10, validation_data=(enc_val, dec_val))

Epoch 1/10
4400/4400 [==============================] - 99s 22ms/step - loss: 3.2191 - val_loss: 2.9144
Epoch 2/10
4400/4400 [==============================] - 94s 21ms/step - loss: 2.6984 - val_loss: 2.6749
Epoch 3/10
4400/4400 [==============================] - 94s 21ms/step - loss: 2.3414 - val_loss: 2.5464
Epoch 4/10
4400/4400 [==============================] - 94s 21ms/step - loss: 2.0518 - val_loss: 2.4777
Epoch 5/10
4400/4400 [==============================] - 94s 21ms/step - loss: 1.8231 - val_loss: 2.4509
Epoch 6/10
4400/4400 [==============================] - 94s 21ms/step - loss: 1.6461 - val_loss: 2.4531
Epoch 7/10
4400/4400 [==============================] - 94s 21ms/step - loss: 1.5086 - val_loss: 2.4621
Epoch 8/10
4400/4400 [==============================] - 94s 21ms/step - loss: 1.4012 - val_loss: 2.4851
Epoch 9/10
4400/4400 [==============================] - 95s 22ms/step - loss: 1.3197 - val_loss: 2.5219
Epoch 10/10
4400/4400 [==============================] - 94s 21m

In [36]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [70]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> you know i m never gonna leave you <end> '

# 회고

## 이번 프로젝트에서 **어려웠던 점**
- 처음에 학습 중 loss가 이상하게 낮아 원인을 찾느라 고생하였습니다. 원인은 토큰화 할 때 토큰의 개수를 15 이하로 하라는 것을 이해하지 못하여 len(sentence)가 15 이하인 경우로 생각하여 생긴 것이었습니다.
- val_loss의 값이 2.2 이하로 내려가지 않아 rnn 층을 하나 더 쌓아 학습을 진행해보았으나 loss의 큰 차이가 없었습니다.